In [1]:
import sys
sys.path.append("../dist")

from element import *
from aes import *

pt = bytes.fromhex('f085c01fefa2af35326467f6facfcf50')
ct = bytes.fromhex('a016124ed2b337a845ca03be0dd014cd')

# What is `Element`?

- A quick verification shows that `Element` implements a group $G$ of size $252$
- A [quick search](https://groupprops.subwiki.org/wiki/Groups_of_order_252) shows that $G$ is soluble
- Compute the derived subgroup $G'$ to be generated from $(1,0,0,0)$, $(0,1,0,0)$ and $(0,0,1,2)$
- Compute $G''$ to be $\{1\}$, so $G'$ is abelian (and $G \cong C_3 \times C_{21}$)
- Compute $G \backslash G'$ to notice that each coset in $G/G' \cong C_4$ and a unique representative in $Q = \langle(0,0,1,1)\rangle$ 
    - Notice that $G' \triangleleft G$, $Q\cap G'$ and $G'Q = G$. So $G = G' \ltimes Q  \cong (C_3 \times C_{21}) \ltimes C_4$

In [2]:
from itertools import product

def to_tuple(x):
    a, x = x % 3, x // 3
    b, x = x % 3, x // 3
    c, x = x % 7, x // 7
    d = x
    return (a,b,c,d)

def from_tuple(t):
    a,b,c,d = t
    return ((d*7 + c)*3 + b)*3 + a

In [3]:
# Maps an element with its inverse
invmap = {Element(i): next(
        Element(j) 
        for j in range(Element.sz) 
        if (Element(j) + Element(i)).to_byte() == 0
    ) for i in range(Element.sz)}

def commutators(tuples):
    """
    Compute all commutators, an approximation for derived subgroup
    """
    alle = [Element(from_tuple(x)) for x in tuples]
    com = [x+y+invmap[x]+invmap[y] for x in alle for y in alle]
    return set([to_tuple(c.to_byte()) for c in com])

def is_normal(tuples):
    """
    Check if group is normal
    """
    A = [Element(i) for i in range(252)]
    B = [Element(from_tuple(x)) for x in tuples]
    com = []
    for x in A:
        for y in B:
            if (x+y+invmap[x]) not in B:
                return False
    return True

# G' turns out to be (i,j,k,k*2)
G_derived = commutators([(i,j,k,l)
    for i,j,k,l in product(*map(range, [3,3,7,4]))
])

# G'' turns out to be {1}
assert {(0,0,0,0)} == commutators(G_derived)

In [4]:
# Get representatives of each coset G/G'
rep_C4 = [Element(from_tuple((0,0,1,1)))*i for i in range(4)]
C4_to_quo = {
    # Maps C4 -> G/G'
    i: set([q + Element(from_tuple(g)) for g in G_derived]) for i,q in enumerate(rep_C4)
}
G_to_C4 = {
    # Maps G -> G/G' -> C4
    g: i for i,quo in C4_to_quo.items() for g in quo
}
# Check that our representatives covered all cosets
assert len(set(sum(map(list, C4_to_quo.values()), []))) == Element.sz

# Solution

- The AES implementation has no substitution operation, so it becomes trivial if the group operation is abelian
- But $G$ isn't abelian

## Strategy
- Solve for `key`=$k$ in $G/G' \cong C_4$ which is abelian (gaussian elimination). Now we know which quotient each byte of $k$ is in.

In [5]:
# We modify the AES implementation for your purpose

# RCON for C4
RCON = [*map(Element, (
    0x00, 0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40,
    0x80, 0x1B, 0x36, 0x6C, 0xD8, 0xAB, 0x4D, 0x9A,
    0x2F, 0x5E, 0xBC, 0x63, 0xC6, 0x97, 0x35, 0x6A,
    0xD4, 0xB3, 0x7D, 0xFA, 0xEF, 0xC5, 0x91, 0x39,
))]
N_ROUNDS = 6
N_BYTES = 16

def shift_rows(s):
    s[0][1], s[1][1], s[2][1], s[3][1] = s[1][1], s[2][1], s[3][1], s[0][1]
    s[0][2], s[1][2], s[2][2], s[3][2] = s[2][2], s[3][2], s[0][2], s[1][2]
    s[0][3], s[1][3], s[2][3], s[3][3] = s[3][3], s[0][3], s[1][3], s[2][3]
    
def add_round_key(s, k):
    for i in range(4):
        for j in range(4):
            s[i][j] += k[i][j]
            
def mix_single_column(a):
    b1, b2, b3, b4 = (
        2*a[0] + 3*a[1] + 1*a[2] + 1*a[3],
        1*a[0] + 2*a[1] + 3*a[2] + 1*a[3],
        1*a[0] + 1*a[1] + 2*a[2] + 3*a[3],
        3*a[0] + 1*a[1] + 1*a[2] + 2*a[3]
    )
    a[0], a[1], a[2], a[3] = b1, b2, b3, b4
    
def mix_columns(s):
    for i in range(4):
        mix_single_column(s[i])
        
def xor_bytes(a, b):
    return [i+j for i, j in zip(a, b)]

def bytes2matrix(text):
    return [text[i:i+4] for i in range(0, len(text), 4)]

def matrix2bytes(matrix):
    return sum(matrix, [])
        
def expand_key(map_to_F, master_key):
    
    key_columns = bytes2matrix(master_key)
    iteration_size = len(master_key) // 4
    rcon = [map_to_F(i) for i in RCON]

    i = 1
    while len(key_columns) < (N_ROUNDS + 1) * 4:
        # Copy previous word.
        word = list(key_columns[-1])

        # Perform schedule_core once every "row".
        if len(key_columns) % iteration_size == 0:
            # Circular shift.
            word.append(word.pop(0))
            # XOR with first byte of R-CON, since the others bytes of R-CON are 0.
            word[0] += rcon[i]
            i += 1

        # XOR with equivalent word from previous iteration.
        word = xor_bytes(word, key_columns[-iteration_size])
        key_columns.append(word)

    # Group key words in 4x4 byte matrices.
    return [key_columns[4*i : 4*(i+1)] for i in range(len(key_columns) // 4)]

def encrypt_block(map_to_F, key, plaintext):

    assert len(plaintext) == N_BYTES

    plain_state = bytes2matrix(plaintext)
    round_keys = expand_key(map_to_F, key)
    
    add_round_key(plain_state, round_keys[0])

    for i in range(1, N_ROUNDS):
        shift_rows(plain_state)
        mix_columns(plain_state)
        add_round_key(plain_state, round_keys[i])

    shift_rows(plain_state)
    add_round_key(plain_state, round_keys[-1])

    return matrix2bytes(plain_state)

In [6]:
# First solve in G/G' ~ C4

C4 = Zmod(4)
element_to_C4 = lambda b: C4(G_to_C4[b])
bytes_to_C4 = lambda bs: [element_to_C4(Element(b)) for b in bs]

# Map pt-ct pair to C4
pt_c4,ct_c4 = [*map(bytes_to_C4, (pt, ct))]
# Create symbolic key
ksym_c4 = PolynomialRing(C4, ["k%d"%d for d in range(16)]).gens()
# Symbolically compute ct
ctsym_c4 = encrypt_block(element_to_C4, ksym_c4, pt_c4)

# Solve for key in quotient C4
mat = matrix(C4, [
    [r[k] for k in ksym_c4] for r in ctsym_c4
])
c1mod = vector(C4, [c-ctsym_c4[i].constant_coefficient() for i,c in enumerate(ct_c4)])
k_c4 = [rep_C4[i] for i in mat.solve_right(c1mod)]

In [7]:
# Compute kernel
F = ZZ^16
M = F/(4*F)
A = mat.T
phi = M.hom([M(a) for a in A])

# Assert that this is the only solution
assert len([M(b) for b in phi.kernel().gens()]) == 0

## Strategy

- Each byte of `ct` can be expressed as a linear sum of bytes of $k$ and constants
    - E.g., $\mathrm{ct}[0] = c_0 = k_0 + 192 + k_2 + \ldots$
    - Note this sum is very long because $G$ is non-abelian and we can't just group the terms together
- Each element of $g \in G$ can be expressed as $g = g' \overline{g}$ where $g' \in G'$ and $\overline{g} \in Q$
- We express each byte of `ct` as $c'\overline{c}$. $c' \in G'$, with $\overline{c} \in Q$ which is known. Since $G'$ is abelian, we can simplify and gaussian eliminate over $G'$
    - E.g., Switching to multiplicative notation, suppose $c_0 = abcd\ldots$. Then denoting $xyx^{-1} = y^x$, we have:

$$
\begin{align}
c_0 
&= a'\overline{a}\quad b'\overline{b}\quad c'\overline{c}\quad d'\overline{d} \quad \ldots \\
&= a' (b'^{\overline{a}}) \overline{a}\overline{b} \quad c'\overline{c}\quad d'\overline{d} \quad \ldots \\
&= a' (b'^{\overline{a}})  (c'^{\overline{a}\overline{b}}) \overline{a}\overline{b}\overline{c}\quad d'\overline{d} \quad \ldots \\
&= a' (b'^{\overline{a}})  (c'^{\overline{a}\overline{b}}) (d'^{\overline{a}\overline{b}\overline{c}})\overline{a}\overline{b}\overline{c}\overline{d} \quad \ldots \\
\end{align}
$$

Since for $x \in G'$, $g \in G$, we have $x^g \in G'$ since $G' \triangleleft G$, this procedure effectively shifts all $G'$ components of $c_0$ to the left. The $Q$ components ($\overline{a}\overline{b}\overline{c}\overline{d}\ldots$) are all known from the previous step. And each term in the $G'$ components is either a known constant, or of the form $k_{n}^q$ where $k_n$ is a byte in `key`, and $q \in Q$. Hence there are a maximum of $|Q| \times 16 = 64$ unknown terms in the $G'$ component.

We first extract out a "trace" of the encryption and compute both $G'$ and $Q$ components of `ct`:

In [8]:
import numpy as np
from functools import reduce, lru_cache, cached_property

class Symbol:
    
    """Shitty class to represent a variable"""
    
    def __init__(self, x, _islist=False):
        if _islist:
            self.add = x
            return
        assert not isinstance(x, int)
        self.add = [x]
    
    def __add__(self, other):
        if isinstance(self.add[-1], Element) and isinstance(other.add[0], Element):
            new = self.add[:-1] + [self.add[-1] + other.add[0]] + other.add[1:]
        else:
            new = self.add + other.add
        return Symbol(new, True)
    
    def __mul__(self, n):
        assert n >= 0
        if n == 0:
            return Symbol([], True)
        return self + (self * (n-1))
    
    def __rmul__(self, n):
        return self*n

ct_sym = encrypt_block(Symbol, [Symbol("k%d"%i) for i in range(16)], [Symbol(Element(x)) for x in pt])

In [9]:
# Seperate each term of the addition into aa' (where a in G', a' in Q)
ct_bsep = []
for cc in ct_sym:
    l = cc.add
    c = []
    for b in l:
        if isinstance(b, str):
            c.append((b, k_c4[int(b[1:])]))
            continue
        r = rep_C4[G_to_C4[b]]
        c.append((b + invmap[r], r))
    ct_bsep.append(c)

In [10]:
# Seperate each ct byte into aa' (where a in G', a' in Q)
ct_sep = []
for l in ct_bsep:

    x,y = l[0]
    sep = [[(Element(0), x)], y]
    for x,y in l[1:]:
        ns,rs = sep
        assert isinstance(rs, Element)
        if isinstance(x, Element):
            sep[1] += y
            ns.append((Element(0), rs + x + invmap[rs]))
            continue
        ns.append((rs,x))
        sep[1] += y
        
    # Make sure we seperated correctly
    assert len(set([x for x,y in sep[0]])) <= 4
    ct_sep.append(sep)

In [11]:
from collections import Counter

# Since G' is abelian, we can permute the elements, so lets simplify the expressions

nct_sep = []
for cc in ct_sep:
    norm, rem = cc
    const = Element(0)
    kvar = []
    for (a,b), coeff in Counter(norm).items():
        if isinstance(b, Element):
            assert a.to_byte() == 0
            const += b * coeff
            continue
        kvar.append(((a,b), coeff))
    nct_sep.append(((kvar, const), rem))

In [12]:
# Now we map everything to G' ~ C21 x C3
ct_C21C3 = [Element(c) + invmap[rem] + invmap[const] for c,((_,const),rem) in zip(ct, nct_sep)]
# Each item on kvars represents one of 64 possible q k_n q^-1
kvars = {
    (a,b): f"g_k{f'{int(b[1:]):02d}'}_{element_to_C4(a)}" for ((kvar, _), _) in nct_sep for (a,b), _ in kvar
}
invkvars = {
    i:j for j,i in kvars.items()
}

## Strategy

- What's left is to solve the $G' \cong C_3 \times C_{27}$ component. 
- Implementation-wise it is easier to solve in $\langle (0,0,1,2)\rangle \cong C_7$ and $\langle(1,0,0,0), (0,1,0,0)\rangle \cong C_3^2$. 
- We can do that as $\langle (0,0,1,2)\rangle \triangleleft G$ and $\langle(1,0,0,0), (0,1,0,0)\rangle \triangleleft G$ which will be explained later

In [13]:
assert is_normal([(0,0,k,(k*2)%4)
    for k in range(7)
])
assert is_normal([(i,j,0,0)
    for i in range(3)
    for j in range(3)
])

## Strategy

Now we try to solve in $C_7$.

We've expressed each byte in `ct` ($c_n$) as $\displaystyle c_n = \sum_{q_i \in Q, \; c \in \mathbb{N}} c q_i k_i' q_i^{-1} + q, \; q \in Q$.
We define an epimorphism $\phi: G' \rightarrow \langle (0,0,1,2)\rangle \cong C_7$, so equating $\phi(c_n)$ with the given ciphertext gives us $16$ linear equations with $64$ unknowns in $C_7$ of the form $\phi(q_i k_i' q_i^{-1})$. We need to get $48$ more equations.

Since as we've verified, $\langle (0,0,1,2)\rangle \triangleleft G$, the action of $Q$ on $\langle (0,0,1,2)\rangle$ by conjugation is well-defined, and maps $Q$ into a subgroup of $\mathrm{Aut}(\langle (0,0,1,2)\rangle) \cong \mathbb{Z}_7^* \cong C_6$ which is abelian. This abelian nature of the action allows us to define a further $48$ linear relations on $\phi(q_i k_i' q_i^{-1})$.

In [14]:
# Now we solve in C7

C7 = Zmod(7)
C7_to_element = lambda i: Element(from_tuple((0,0,i%7,((i%7)*2)%4)))
element_to_C7 = lambda x: (lambda x: x[2])(to_tuple((x + invmap[rep_C4[element_to_C4(x)]]).to_byte()))

ksym_varlist = sorted(PolynomialRing(C7, [*kvars.values()]).gens(), key=lambda x: str(x))
ksym_c7 = {str(i): i for i in ksym_varlist}
ctsym_c7 = [
    sum([coeff*ksym_c7[kvars[(a,b)]] for (a,b), coeff in kvar], 0)
    for (kvar, _), _ in nct_sep
]
ct_C7 = [element_to_C7(i) for i in ct_C21C3]

# Group action constraints:
# g -> x in C7
# a -> y in C4
# aga^-1 -> x + x * 5   if y%2 mod 1 else x
# g_k<a>_<b> - g_k<a>_0*6 == 0  if b%2 mod 1 else   g_k<a>_<b> - g_k<a>_0 == 0
for kidx in range(16):
    for oidx in range(1,4):
        gb = ksym_c7[f"g_k{kidx:02d}_{oidx}"]
        x = ksym_c7[f"g_k{kidx:02d}_0"]
        if oidx % 2 == 1:
            ctsym_c7.append(gb - 6*x)
        else:
            ctsym_c7.append(gb - x)
        ct_C7.append(0)

mat = matrix(C7, [
    [r[k] for k in ksym_varlist] for r in ctsym_c7
])
sol = mat.solve_right(ct_C7)
k_c7 = [sol[ksym_varlist.index(ksym_c7[f"g_k{kidx:02d}_0"])] for kidx in range(16)]
k_c7 = [Element(from_tuple((0,0,int(i),(int(i)*2)%4))) for i in k_c7]

assert len(mat.right_kernel().basis()) == 0

/Tools/sage/local/var/lib/sage/venv-python3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DeprecationWarning: solve_right should be called with a vector or matrix
See http://trac.sagemath.org/17405 for details.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Strategy

Now we do the same for solve in $C_3^2$, defining a map $G' \rightarrow \langle(1,0,0,0), (0,1,0,0)\rangle = X \cong C_3^2$. Since we have $X \triangleleft G$, the action of $Q$ on $X$ by conjugation is well defined, and maps $Q \rightarrow \mathrm{Aut}(X) \cong \mathrm{GL}_2(3)$ on which we similarly define the additional $48$ constraints.

In [15]:
# Now we solve in the first C3 x C3

C3 = Zmod(3)
C33_to_element = lambda i: Element(from_tuple((i[0], i[1] ,0,0)))
element_to_C33 = lambda x: (lambda x: (C3(x[0]),C3(x[1])))(to_tuple(x.to_byte()))

_ = [*kvars.values()]
_ = [i + f"_{j}" for i in _ for j in range(2)]
ksym_varlist = sorted(PolynomialRing(C3, _).gens(), key=lambda x: str(x))
ksym = {str(i): i for i in ksym_varlist}
ctsym = [
    (
        sum([coeff*ksym[kvars[(a,b)] + "_0"] for (a,b), coeff in kvar], 0), 
        sum([coeff*ksym[kvars[(a,b)] + "_1"] for (a,b), coeff in kvar], 0))
    for (kvar, _), _ in nct_sep
]
ct_C33 = [element_to_C33(i) for i in ct_C21C3]

# g -> (x,y) in C33
# a -> z in C4 
# g_<b> =
#    x*(1,2) + y*(2,2)        b == 1
#    2*g_<0>                  b == 2
#    2*x*(1,2) + 2*y*(2,2)    b == 3
for kidx in range(16):
    for oidx in range(1,4):
        a,b = ksym[f"g_k{kidx:02d}_{oidx}_0"], ksym[f"g_k{kidx:02d}_{oidx}_1"]
        x,y = ksym[f"g_k{kidx:02d}_0_0"], ksym[f"g_k{kidx:02d}_0_1"]
        if oidx == 1:
            ctsym.append((
                a - (1*x + 2*y),
                b - (2*x + 2*y)
            ))
            ct_C33.append((0,0))
        if oidx == 2:
            ctsym.append((
                a - (2*x),
                b - (2*y)
            ))
            ct_C33.append((0,0))
        if oidx == 3:
            ctsym.append((
                a - (2*x + 4*y),
                b - (4*x + 4*y)
            ))
            ct_C33.append((0,0))
        
mat = matrix(C3, [
    [x[k] for k in ksym_varlist] for r in ctsym for x in r
])
ct_C33_mat = [x for r in ct_C33 for x in r]

sol = mat.solve_right(ct_C33_mat)
k_c33 = [(
    sol[ksym_varlist.index(ksym[f"g_k{kidx:02d}_0_0"])],
    sol[ksym_varlist.index(ksym[f"g_k{kidx:02d}_0_1"])]
) for kidx in range(16)]
k_c33 = [Element(from_tuple((int(i[0]),int(i[1]),0,0))) for i in k_c33]

assert len(mat.right_kernel().basis()) == 0

## Strategy

At this point, we have solved for `key` in $G'$ and in $G/G'$ so it's trivial reconstruct `key`:

In [16]:
from aes import encrypt_block as encblk
key_recovered = bytes([(z + x + y).to_byte() for x,y,z in zip(k_c7, k_c4, k_c33)])
assert bytes(encblk(key_recovered, pt)) == bytes(ct)

print("flag:", "SEE{%s}"%key_recovered.hex())

flag: SEE{143349be7827dc5f9916f4adb97e6241}


...

...

...

...

...

# Scratchpad below onwards, can ignore

In [ ]:
print([to_tuple((x + y).to_byte())[2:] for x,y in zip(k_c7, k_c4)])
print([to_tuple(k)[2:] for k in key])

In [15]:
_kmap = [Element(i) for i in key]
_kmap = [b + invmap[rep_C4[G_to_C4[b]]] for b in _kmap]
_nkmap = [(lambda a,b: a + _kmap[int(b[1:])] + invmap[a])(*invkvars[str(v)]) for v in ksym_varlist]
_nkmap = vector(C7, [element_to_C7(i) for i in _nkmap])
print((mat*_nkmap)[:16]), print(ct_C7[:16])
print((mat*_nkmap)[16:]), print(ct_C7[16:])

(6, 3, 4, 5, 5, 4, 5, 1, 0, 5, 2, 3, 3, 2, 6, 2)
[6, 3, 4, 5, 5, 4, 5, 1, 0, 5, 2, 3, 3, 2, 6, 2]
(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


(None, None)

In [284]:
_kmap = [Element(i) for i in key]
_kmap = [b + invmap[rep_C4[G_to_C4[b]]] for b in _kmap]
#print([
#    sum([x+(y if isinstance(y, Element) else _kmap[int(y[1:])])+invmap[x] for x,y in norm], Element(0)) + rem
#    for norm, rem in ct_sep
#])
print([
    sum([coeff*(a + _kmap[int(b[1:])] + invmap[a]) for (a,b), coeff in kvar], Element(0)) + const + rem
    for (kvar, const), rem in nct_sep
])
print([*map(Element, ct)])

[<E:248>, <E:126>, <E:32>, <E:218>, <E:227>, <E:197>, <E:136>, <E:89>, <E:190>, <E:76>, <E:189>, <E:197>, <E:158>, <E:221>, <E:157>, <E:211>]
[<E:248>, <E:126>, <E:32>, <E:218>, <E:227>, <E:197>, <E:136>, <E:89>, <E:190>, <E:76>, <E:189>, <E:197>, <E:158>, <E:221>, <E:157>, <E:211>]


In [181]:
e_to_a = {i: [element_to_C33(r + i + invmap[r]) for r in rep_C4] for i in map(Element, range(252))}
c33_to_a = {}
for i,l in e_to_a.items():
    c = tuple(element_to_C33(i))
    if c not in c33_to_a: c33_to_a[c] = set()
    c33_to_a[c].add(tuple(map(tuple, [C33(x) -  2*c[0]*C33((1,2)) - 2*c[1]*C33((2,2)) for x in l])))
c33_to_a

{(0, 0): {((0, 0), (0, 0), (0, 0), (0, 0))},
 (1, 0): {((2, 2), (2, 1), (0, 2), (0, 0))},
 (2, 0): {((1, 1), (1, 2), (0, 1), (0, 0))},
 (0, 1): {((2, 0), (1, 1), (2, 1), (0, 0))},
 (1, 1): {((1, 2), (0, 2), (2, 0), (0, 0))},
 (2, 1): {((0, 1), (2, 0), (2, 2), (0, 0))},
 (0, 2): {((1, 0), (2, 2), (1, 2), (0, 0))},
 (1, 2): {((0, 2), (1, 0), (1, 1), (0, 0))},
 (2, 2): {((2, 1), (0, 1), (1, 0), (0, 0))}}

In [88]:
C33((0,2)) + C33((1,1))

(1, 0)

In [ ]:
# g -> (x,y) in C33
# a -> z in C4 
# g_<b> =
#    x*(1,2) + y*(2,2)    b == 1
#    2*g_<0>   b == 2
#    2*x*(1,2) + 2*y*(2,2)    b == 3

In [354]:
e_to_a = {i: [element_to_C7(r + i + invmap[r]) for r in rep_C4] for i in map(Element, range(252))}
c7_to_a = {}
for i,l in e_to_a.items():
    c = element_to_C7(i)
    if c not in c7_to_a: c7_to_a[c] = set()
    c7_to_a[c].add(tuple([C7(x-c) for x in l]))
c7_to_a

{0: {(0, 0, 0, 0)},
 1: {(0, 5, 0, 5)},
 2: {(0, 3, 0, 3)},
 3: {(0, 1, 0, 1)},
 4: {(0, 6, 0, 6)},
 5: {(0, 4, 0, 4)},
 6: {(0, 2, 0, 2)}}

In [ ]:
# g -> x in C7
# a -> y in C4
# aga^-1 -> x + x * 5   if y%2 mod 1 else x

In [248]:
# Now we solve in C21

C21 = Zmod(21)
C21_to_element = lambda i: Element(from_tuple((0,i%3,i%7,((i%7)*2)%4)))
element_to_C21 = lambda x: (lambda x: _[(0,x[1],x[2],x[3])])(to_tuple((x + invmap[rep_C4[element_to_C4(x)]]).to_byte()))

ksym_varlist = PolynomialRing(C21, [*kvars.values()]).gens()
ksym_c21 = {str(i): i for i in ksym_varlist}
ctsym_c21 = [
    sum([coeff*ksym_c21[kvars[(a,b)]] for (a,b), coeff in kvar], 0)
    for (kvar, _), _ in nct_sep
]
ct_C21 = [element_to_C21(i) for i in ct_C21C3]

mat = matrix(C21, [
    [r[k] for k in ksym_varlist] for r in ctsym_c21
])

_kmap = [Element(i) for i in key]
_kmap = [b + invmap[rep_C4[G_to_C4[b]]] for b in _kmap]
_nkmap = [(lambda a,b: a + _kmap[int(b[1:])] + invmap[a])(*invkvars[str(v)]) for v in ksym_varlist]
_nkmap = vector(C21, [element_to_C21(i) for i in _nkmap])
mat*_nkmap, ct_C21

((3, 14, 6, 19, 14, 9, 3, 7, 15, 10, 16, 5, 17, 14, 13, 12),
 [3, 14, 6, 19, 14, 9, 3, 7, 15, 10, 16, 5, 17, 14, 13, 12])

In [260]:
e_to_a = {i: [element_to_C21(r + i + invmap[r]) for r in rep_C4] for i in map(Element, range(252))}
c21_to_a = {}
for i,l in e_to_a.items():
    c = element_to_C21(i)
    if c not in c21_to_a: c21_to_a[c] = set()
    c21_to_a[c].add(tuple(l))
c21_to_a

{0: {(0, 0, 0, 0), (0, 7, 0, 14), (0, 14, 0, 7)},
 7: {(7, 0, 14, 0), (7, 7, 14, 14), (7, 14, 14, 7)},
 14: {(14, 0, 7, 0), (14, 7, 7, 14), (14, 14, 7, 7)},
 15: {(15, 6, 15, 6), (15, 13, 15, 20), (15, 20, 15, 13)},
 1: {(1, 6, 8, 6), (1, 13, 8, 20), (1, 20, 8, 13)},
 8: {(8, 6, 1, 6), (8, 13, 1, 20), (8, 20, 1, 13)},
 9: {(9, 5, 9, 19), (9, 12, 9, 12), (9, 19, 9, 5)},
 16: {(16, 5, 2, 19), (16, 12, 2, 12), (16, 19, 2, 5)},
 2: {(2, 5, 16, 19), (2, 12, 16, 12), (2, 19, 16, 5)},
 3: {(3, 4, 3, 11), (3, 11, 3, 4), (3, 18, 3, 18)},
 10: {(10, 4, 17, 11), (10, 11, 17, 4), (10, 18, 17, 18)},
 17: {(17, 4, 10, 11), (17, 11, 10, 4), (17, 18, 10, 18)},
 18: {(18, 3, 18, 3), (18, 10, 18, 17), (18, 17, 18, 10)},
 4: {(4, 3, 11, 3), (4, 10, 11, 17), (4, 17, 11, 10)},
 11: {(11, 3, 4, 3), (11, 10, 4, 17), (11, 17, 4, 10)},
 12: {(12, 2, 12, 16), (12, 9, 12, 9), (12, 16, 12, 2)},
 19: {(19, 2, 5, 16), (19, 9, 5, 9), (19, 16, 5, 2)},
 5: {(5, 2, 19, 16), (5, 9, 19, 9), (5, 16, 19, 2)},
 6: {(6, 1, 6